In [ ]:
print('OK')


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_community.llms import CTransformers
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os


In [ ]:
os.environ['PINECONE_API_KEY'] = 'YOUR_API_KEY'
PINECONE_API_KEY='YOUR_API_KEY'
PINCONE_API_ENV = 'quickstart'

In [ ]:
#Extract data from pdf
def load_pdf(data):
   loader= DirectoryLoader(data,glob='*.pdf',loader_cls=PyMuPDFLoader)

   documents=loader.load()  
   return documents

In [ ]:
extracted_data = load_pdf('data/')

In [ ]:
#Covert Data into chunks | Create text chunks

def textt_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks



In [ ]:
text_chunks = textt_split(extracted_data)
print(len(text_chunks))


In [ ]:
#download embedding model 
def download_hugging_face_embedding():
    embedding= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [ ]:
embedding=download_hugging_face_embedding()

In [ ]:
embedding

In [ ]:
querry_result=embedding.embed_query("Hello World")
print("Length of the query result",len(querry_result))

In [ ]:
querry_result


In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="chatbot"
pc.list_indexes()
# docsearc=PineconeVectorStore.from_texts([t.page_content for t in text_chunks],embedding,index_name=index_name)

In [ ]:
#if we already have an index we can load the data 
docsearch=PineconeVectorStore.from_existing_index(index_name,embedding)
querry="What are Allergies"
docs=docsearch.similarity_search(querry,k=3)
print('Result',docs)

In [ ]:
prompt_template="""
use the following piece of information to answer the user's question.
If you dont know the answer, just say "I don't know", dont make up information.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful Answer: 
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template,input_vaiables=['context','question'])
chain_type_kwargs={'prompt':PROMPT}

In [ ]:
config = {'max_new_tokens': 256, 'repetition_penalty': 0.8}
llm = CTransformers(model='model//llama-2-7b-chat.ggmlv3.q4_0.bin',model_type='llama',config=config)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
while True:
    user_input = input("Ask a question: ")

    if user_input == 'exit':
        break
    result = qa({'query': user_input})
    print(result['result'])